In [1]:
import requests
import pandas as pd
from datetime import datetime
import os
import gzip
import shutil
import pandas as pd
from sklearn.impute import KNNImputer
from datetime import datetime
import requests
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date, trim
import requests
import pandas as pd
from datetime import datetime
from pyspark.sql.functions import col, count, when, isnan
from datetime import timedelta
from pyspark.sql import Window
from pyspark.sql.functions import last

In [2]:
#Diable the warnings
import warnings
warnings.filterwarnings('ignore')

In [31]:
# FMP API key
api_key = '47461a32a9bcf3b0a6c2c6e98ccfab42'

# Symbol for EURO STOXX 50
stock_symbol = '^STOXX50E'

# URL for the FMP historical data endpoint
url = f'https://financialmodelingprep.com/api/v3/historical-price-full/{stock_symbol}?from=2005-02-01&to={datetime.now().strftime("%Y-%m-%d")}&apikey={api_key}'

# FMP API request
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Convert the response to JSON
    data = response.json()

    # Extract the historical data
    if 'historical' in data:
        historical_data = data['historical']

        # Convert to a pandas DataFrame
        index_data = pd.DataFrame(historical_data)

        # Display the DataFrame
        print(index_data)
    else:
        print("No 'historical' data found in the response.")
else:
    print(f'Failed to retrieve data: {response.status_code}')


            date         open         high          low        close  \
0     2024-01-12  4453.240230  4492.770020  4453.240230  4480.020020   
1     2024-01-11  4481.399900  4510.009770  4441.459960  4442.279790   
2     2024-01-10  4467.149900  4484.799800  4459.540040  4468.979980   
3     2024-01-09  4486.990230  4486.990230  4446.620120  4467.169920   
4     2024-01-08  4461.799800  4487.259770  4438.490230  4485.479980   
5     2024-01-05  4458.740230  4474.879880  4418.080080  4463.509770   
6     2024-01-04  4451.720210  4476.720210  4443.419920  4474.009770   
7     2024-01-03  4514.200200  4515.700200  4433.140140  4448.129880   
8     2024-01-02  4528.450200  4568.799800  4484.819820  4512.810060   
9     2023-12-29  4518.120120  4536.439940  4517.310060  4521.649900   
10    2023-12-28  4536.680180  4542.750000  4512.640140  4514.379880   
11    2023-12-27  4521.799800  4542.330080  4520.509770  4528.379880   
12    2023-12-22  4520.600100  4528.049800  4505.729980  4521.47

In [33]:
# Sort index_data by 'date' in ascending order and display
sorted_index_data = index_data.sort_values(by='date', ascending=True)
print(sorted_index_data)

            date         open         high          low        close  \
4790  2005-02-01  3008.850098  3008.850098  3008.850098  3008.850098   
4789  2005-02-02  3022.340088  3022.340088  3022.340088  3022.340088   
4788  2005-02-03  3010.389893  3010.389893  3010.389893  3010.389893   
4787  2005-02-04  3037.139893  3037.139893  3037.139893  3037.139893   
4786  2005-02-07  3051.969971  3051.969971  3051.969971  3051.969971   
4785  2005-02-08  3055.899902  3055.899902  3055.899902  3055.899902   
4784  2005-02-09  3044.360107  3044.360107  3044.360107  3044.360107   
4783  2005-02-10  3044.000000  3044.000000  3044.000000  3044.000000   
4782  2005-02-11  3080.080078  3080.080078  3080.080078  3080.080078   
4781  2005-02-14  3075.760010  3075.760010  3075.760010  3075.760010   
4780  2005-02-15  3086.949951  3086.949951  3086.949951  3086.949951   
4779  2005-02-16  3068.550049  3068.550049  3068.550049  3068.550049   
4778  2005-02-17  3067.340088  3067.340088  3067.340088  3067.34

In [4]:
index_data.describe()

,open,high,low,close,adjClose,volume,unadjustedVolume,change,changePercent,vwap,changeOverTime
count,4791.000000,4791.000000,4791.000000,4791.000000,4791.000000,4.791000e+03,4.791000e+03,4791.000000,4791.000000,4791.000000,4791.000000
mean,3320.059467,3340.867033,3297.465889,3319.823072,3319.822443,2.353284e+07,2.353840e+07,-0.236395,-0.002130,1908.423309,-0.000021
std,588.594719,586.599499,591.306460,588.948901,588.947796,2.573622e+07,2.573401e+07,35.682655,1.176778,1773.748300,0.011768
min,1812.780029,1823.250000,1765.489990,1809.979981,1809.979981,0.000000e+00,0.000000e+00,-338.020020,-11.724000,0.000000,-0.117240
25%,2901.079956,2925.299927,2879.625000,2902.965088,2902.965088,0.000000e+00,0.000000e+00,-14.775025,-0.443000,0.000000,-0.004430
50%,3318.399902,3335.399902,3298.469971,3320.149902,3320.149902,2.380700e+07,2.381400e+07,0.000000,0.000000,2978.636720,0.000000
75%,3702.000000,3722.234985,3680.810059,3703.500000,3703.500000,4.038135e+07,4.038135e+07,16.145020,0.499000,3483.370035,0.004990
max,4555.779785,4593.109860,4542.109860,4557.569824,4557.569824,2.358453e+08,2.358453e+08,246.070070,10.076000,4555.620000,0.100760


In [5]:
# FMP API key
api_key = '47461a32a9bcf3b0a6c2c6e98ccfab42'

# Symbol for the stock (e.g., '^STOXX50E' for EURO STOXX 50)
stock_symbol = '^STOXX50E'

# Specify the time interval (e.g., '5min', '15min', '1hour', '4hour', '1day')
time_interval = '5min'

# Specify the period for RSI calculation
period = 14  # Commonly used period for RSI

# URL for the FMP RSI endpoint
url = f'https://financialmodelingprep.com/api/v3/technical_indicator/{time_interval}/{stock_symbol}?type=rsi&period={period}&apikey={api_key}'

# FMP API request for RSI data
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Convert the response to JSON
    data = response.json()

    # Convert to a pandas DataFrame
    rsi_data = pd.DataFrame(data)

    # Display the DataFrame
    print(rsi_data)
else:
    print(f'Failed to retrieve RSI data: {response.status_code}')


                    date     open     high      low    close  volume  \
0    2024-01-12 17:25:00  4474.40  4476.72  4474.40  4476.07       0   
1    2024-01-12 17:20:00  4474.03  4475.16  4473.65  4475.16       0   
2    2024-01-12 17:15:00  4473.65  4475.14  4472.85  4474.31       0   
3    2024-01-12 17:10:00  4476.31  4477.41  4472.58  4473.53       0   
4    2024-01-12 17:05:00  4473.81  4476.44  4473.15  4476.44       0   
..                   ...      ...      ...      ...      ...     ...   
810  2024-01-03 09:20:00  4511.87  4513.66  4507.06  4507.27       0   
811  2024-01-03 09:15:00  4511.36  4513.35  4509.33  4512.76       0   
812  2024-01-03 09:10:00  4513.57  4515.70  4510.97  4511.66       0   
813  2024-01-03 09:05:00  4509.95  4515.56  4509.64  4513.45       0   
814  2024-01-03 09:00:00  4514.20  4514.20  4508.89  4509.87       0   

           rsi  
0    48.527265  
1    46.931662  
2    45.465397  
3    44.150514  
4    48.174394  
..         ...  
810   0.000000  

# Interest Rate API (Eurostat)

In [6]:
# Dataset code
dataset_code = "ei_mfir_m"

# API URL for downloading the dataset
api_url = f"https://ec.europa.eu/eurostat/api/dissemination/sdmx/2.1/data/{dataset_code}?format=TSV&compressed=true"

# Download the dataset
response = requests.get(api_url, stream=True)
output_file = dataset_code + ".tsv.gz"

if response.status_code == 200:
    with open(output_file, 'wb') as f:
        f.write(response.content)
    print("Dataset downloaded successfully.")
else:
    print(f"Error downloading dataset. Status code: {response.status_code}")

# Unzipping the dataset
with gzip.open(output_file, 'rb') as f_in:
    with open(dataset_code + ".tsv", 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

print("Dataset unzipped successfully.")

Dataset downloaded successfully.
Dataset unzipped successfully.


In [7]:
# Get the current working directory
current_directory = os.getcwd()
print("Current Working Directory:", current_directory)

# Construct the file path dynamically
file_name = 'ei_mfir_m.tsv'  # File Name
file_path = os.path.join(current_directory, file_name)

# Load the dataset
data = pd.read_csv(file_path, sep='\t')

# Split the first column into separate columns
split_columns = data.iloc[:, 0].str.split(',', expand=True)
split_columns.columns = ['freq', 's_adj', 'p_adj', 'indic', 'geo']

# Combine the split columns with the original data (excluding the first column)
structured_data = pd.concat([split_columns, data.iloc[:, 1:]], axis=1)

# Apply filters
structured_data = structured_data[
    (structured_data['indic'] == 'MF-3MI-RT')
]

# Drop unnecessary columns
structured_data.drop(['freq', 's_adj', 'p_adj'], axis=1, inplace=True)

# Filter for date range (from October 1, 2004, onwards)
date_columns = structured_data.columns[3:]
dates_from_oct_2004 = date_columns[date_columns >= '2005-02-01 ']
structured_data = structured_data[['indic','geo'] + list(dates_from_oct_2004)]

# Reset the index for a clean table
structured_data.reset_index(drop=True, inplace=True)

# Display the structured DataFrame
pd.set_option('display.max_columns', None)

Current Working Directory: /home/fabiopoli/Desktop


In [8]:
# Filter out only the columns with date values for melting
date_columns = structured_data.columns[5:]  
data_for_melting = structured_data[['geo'] + list(date_columns)]

# Melt the DataFrame to change its structure
structured_data = pd.melt(data_for_melting, id_vars=['geo'], var_name='Period', value_name='Interest')

# Filter for 'geo' == 'EA'
Interest_data_ea = structured_data[structured_data['geo'] == 'EA']

# Apply forward filling
Interest_data_ea.loc[:, 'Interest'] = Interest_data_ea['Interest'].ffill()

# Rearrange the columns
Interest_data_ea = Interest_data_ea[['Period', 'geo', 'Interest']]

# Display the transformed DataFrame
pd.set_option('display.max_rows', None)  # This option displays all the rows
print(Interest_data_ea.head(10))


       Period geo  Interest
3    2005-06   EA  2.11100 
16   2005-07   EA  2.11940 
29   2005-08   EA  2.13250 
42   2005-09   EA  2.13910 
55   2005-10   EA  2.19660 
68   2005-11   EA  2.36090 
81   2005-12   EA  2.47290 
94   2006-01   EA  2.51170 
107  2006-02   EA  2.60040 
120  2006-03   EA  2.72260 


In [9]:
# Convert 'Period' to datetime
Interest_data_ea['Period'] = pd.to_datetime(Interest_data_ea['Period'])

# Set 'Period' as the index
Interest_data_ea.set_index('Period', inplace=True)

# Create a daily date range that covers the entire period in your data
start_date = Interest_data_ea.index.min()
end_date = Interest_data_ea.index.max()
all_days = pd.date_range(start=start_date, end=end_date, freq='D')

# Reindex the DataFrame to this date range, forward filling the missing values
daily_interest_data = Interest_data_ea.reindex(all_days).ffill()

# Reset the index so 'Period' becomes a column again
daily_interest_data.reset_index(inplace=True)
daily_interest_data.rename(columns={'index': 'Period'}, inplace=True)

# Display the first 10 rows of the daily data
print(daily_interest_data.head(10))

      Period geo  Interest
0 2005-06-01  EA  2.11100 
1 2005-06-02  EA  2.11100 
2 2005-06-03  EA  2.11100 
3 2005-06-04  EA  2.11100 
4 2005-06-05  EA  2.11100 
5 2005-06-06  EA  2.11100 
6 2005-06-07  EA  2.11100 
7 2005-06-08  EA  2.11100 
8 2005-06-09  EA  2.11100 
9 2005-06-10  EA  2.11100 


In [10]:
# Check if 'Period' is in the columns; if not, the index might already be the 'Period'
if 'Period' in Interest_data_ea.columns:
    Interest_data_ea['Period'] = pd.to_datetime(Interest_data_ea['Period'])
    Interest_data_ea.set_index('Period', inplace=True)
elif not isinstance(Interest_data_ea.index, pd.DatetimeIndex):
    # If the index is not a datetime, convert it
    Interest_data_ea.index = pd.to_datetime(Interest_data_ea.index)

# Create a daily date range
start_date = Interest_data_ea.index.min()
end_date = Interest_data_ea.index.max()
all_days = pd.date_range(start=start_date, end=end_date, freq='D')

# Reindex to create a DataFrame with all days, introducing NaNs for days without data
interest_data = Interest_data_ea.reindex(all_days)

# Convert the index to a numerical value for KNN
interest_data['Day_Ordinal'] = interest_data.index.map(datetime.toordinal)

# Extract the 'Interest' column for imputation
knn_data = interest_data[['Day_Ordinal', 'Interest']].values

# Initialize KNN Imputer
knn_imputer = KNNImputer(n_neighbors=5)

# Apply KNN imputation
knn_data_imputed = knn_imputer.fit_transform(knn_data)

# Put the imputed 'Interest' data back into the DataFrame
interest_data['Interest'] = knn_data_imputed[:, 1]

# Drop the 'Day_Ordinal' column as it's no longer needed
interest_data.drop('Day_Ordinal', axis=1, inplace=True)

# Reset index to make 'Period' a column again
interest_data.reset_index(inplace=True)
interest_data.rename(columns={'index': 'Period'}, inplace=True)

# If the 'geo' column is required, fill it with a constant value (e.g., 'EA')
interest_data['geo'] = 'EA'

# Display the first 10 rows of the daily data
print(interest_data.head(10))

      Period geo  Interest
0 2005-06-01  EA   2.11100
1 2005-06-02  EA   2.13972
2 2005-06-03  EA   2.13972
3 2005-06-04  EA   2.13972
4 2005-06-05  EA   2.13972
5 2005-06-06  EA   2.13972
6 2005-06-07  EA   2.13972
7 2005-06-08  EA   2.13972
8 2005-06-09  EA   2.13972
9 2005-06-10  EA   2.13972


# Bonds Yields API (Eurostat)

In [11]:
# Dataset code
dataset_code = "irt_euryld_d"

# API URL for downloading the dataset
api_url = f"https://ec.europa.eu/eurostat/api/dissemination/sdmx/2.1/data/{dataset_code}?format=TSV&compressed=true"

# Download the dataset
response = requests.get(api_url, stream=True)
output_file = dataset_code + ".tsv.gz"

if response.status_code == 200:
    with open(output_file, 'wb') as f:
        f.write(response.content)
    print("Dataset downloaded successfully.")
else:
    print(f"Error downloading dataset. Status code: {response.status_code}")

# Unzipping the dataset
with gzip.open(output_file, 'rb') as f_in:
    with open(dataset_code + ".tsv", 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

print("Dataset unzipped successfully.")

Dataset downloaded successfully.
Dataset unzipped successfully.


In [12]:
# Get the current working directory
current_directory = os.getcwd()
print("Current Working Directory:", current_directory)

# Construct the file path dynamically
file_name = 'irt_euryld_d.tsv'  # File Name
file_path = os.path.join(current_directory, file_name)

# Load the dataset
data = pd.read_csv(file_path, sep='\t')

# Split the first column into separate columns
split_columns = data.iloc[:, 0].str.split(',', expand=True)
split_columns.columns = ['freq', 'yld_curv', 'maturity', 'bonds', 'geo']

# Combine the split columns with the original data (excluding the first column)
structured_data = pd.concat([split_columns, data.iloc[:, 1:]], axis=1)

# Apply filters
structured_data = structured_data[
    (structured_data['yld_curv'] == 'SPOT_RT') & 
    (structured_data['bonds'] == 'CGB_EA')
]

# Drop unnecessary columns
structured_data.drop(['freq', 'geo'], axis=1, inplace=True)

# Filter for date range (from October 1, 2004, onwards)
date_columns = structured_data.columns[3:]
dates_from_oct_2004 = date_columns[date_columns >= '2004-10-01 ']
structured_data = structured_data[['yld_curv', 'maturity', 'bonds'] + list(dates_from_oct_2004)]

# Reset the index for a clean table
structured_data.reset_index(drop=True, inplace=True)

# Display the structured DataFrame
pd.set_option('display.max_columns', None)

Current Working Directory: /home/fabiopoli/Desktop


In [13]:
# Filter out only the columns with date values for melting
date_columns = structured_data.columns[3:]  # Assuming date columns start from the 4th column
data_for_melting = structured_data[['maturity'] + list(date_columns)]

# Melt the DataFrame to change its structure
bonds_data = pd.melt(data_for_melting, id_vars=['maturity'], var_name='Period', value_name='Rate')

# Specify the desired maturities (M1 to M12 and Y1 to Y30)
desired_maturities = [f'M{i}' for i in range(1, 3)] + [f'Y{i}' for i in range(1, 10)]

# Filter the DataFrame to include only the specified maturities
bonds_data = bonds_data[bonds_data['maturity'].isin(desired_maturities)]

# Rearrange the columns
bonds_data = bonds_data[['Period', 'maturity', 'Rate']]

# Display the transformed DataFrame
pd.set_option('display.max_rows', None)  # This option displays all the rows
print(bonds_data.head())


          Period maturity     Rate
9    2004-10-01        Y1  2.27378
141  2004-10-01        Y2  2.58136
273  2004-10-01        Y3  2.86602
286  2004-10-01        Y4  3.12325
298  2004-10-01        Y5  3.35223


# Inflation Rate API (Eurostat)

In [14]:
# Dataset code
dataset_code = "prc_hicp_manr"

# API URL for downloading the dataset
api_url = f"https://ec.europa.eu/eurostat/api/dissemination/sdmx/2.1/data/{dataset_code}?format=TSV&compressed=true"

# Download the dataset
response = requests.get(api_url, stream=True)
output_file = dataset_code + ".tsv.gz"

if response.status_code == 200:
    with open(output_file, 'wb') as f:
        f.write(response.content)
    print("Dataset downloaded successfully.")
else:
    print(f"Error downloading dataset. Status code: {response.status_code}")

# Unzipping the dataset
with gzip.open(output_file, 'rb') as f_in:
    with open(dataset_code + ".tsv", 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

print("Dataset unzipped successfully.")

Dataset downloaded successfully.
Dataset unzipped successfully.


In [15]:
# Get the current working directory
current_directory = os.getcwd()
print("Current Working Directory:", current_directory)

# Construct the file path dynamically
file_name = 'prc_hicp_manr.tsv'  # File Name
file_path = os.path.join(current_directory, file_name)

# Load the dataset
data = pd.read_csv(file_path, sep='\t')

# Split the first column into separate columns
split_columns = data.iloc[:, 0].str.split(',', expand=True)
split_columns.columns = ['freq', 'unit', 'coicop', 'geo']

# Combine the split columns with the original data (excluding the first column)
structured_data = pd.concat([split_columns, data.iloc[:, 1:]], axis=1)

# Apply filters
structured_data = structured_data[
    (structured_data['unit'] == 'RCH_A') & 
    (structured_data['coicop'] == 'CP00')
]

# Drop unnecessary columns
structured_data.drop(['freq'], axis=1, inplace=True)

# Filter for date range (from January 1, 2005, onwards)
date_columns = structured_data.columns[3:]
dates_from_oct_2004 = date_columns[date_columns >= '2005-02-01 ']
structured_data = structured_data[['unit', 'coicop', 'geo'] + list(dates_from_oct_2004)]

# Reset the index for a clean table
structured_data.reset_index(drop=True, inplace=True)

# Display the structured DataFrame
pd.set_option('display.max_columns', None)

Current Working Directory: /home/fabiopoli/Desktop


In [16]:
# Filter out only the columns with date values for melting
date_columns = structured_data.columns[5:]  # Assuming date columns start from the 6th column
data_for_melting = structured_data[['geo'] + list(date_columns)]

# Melt the DataFrame to change its structure
inflation_data = pd.melt(data_for_melting, id_vars=['geo'], var_name='Period', value_name='Inflation')

# Filter for 'geo' == 'EA'
inflation_data_ea = inflation_data[inflation_data['geo'] == 'EA']

# Apply forward filling
inflation_data_ea.loc[:, 'Inflation'] = inflation_data_ea['Inflation'].ffill()

# Rearrange the columns
inflation_data_ea = inflation_data_ea[['Period', 'geo', 'Inflation']]

# Display the transformed DataFrame
pd.set_option('display.max_rows', None)  # This option displays all the rows
print(inflation_data_ea.head(10))

       Period geo Inflation
9    2005-05   EA      2.0 
54   2005-06   EA      2.0 
99   2005-07   EA      2.1 
144  2005-08   EA      2.2 
189  2005-09   EA      2.6 
234  2005-10   EA      2.5 
279  2005-11   EA      2.3 
324  2005-12   EA      2.3 
369  2006-01   EA      2.4 
414  2006-02   EA      2.4 


In [17]:
# Convert 'Period' to datetime
inflation_data_ea['Period'] = pd.to_datetime(inflation_data_ea['Period'])

# Set 'Period' as the DataFrame index
inflation_data_ea.set_index('Period', inplace=True)

# Resample to daily data using forward fill
daily_data_ffill = inflation_data_ea.resample('D').ffill()

# Reset index to move 'Period' back to a column
daily_data_ffill.reset_index(inplace=True)

# Display the first 10 rows of the daily data
print(daily_data_ffill.head(10))

      Period geo Inflation
0 2005-05-01  EA      2.0 
1 2005-05-02  EA      2.0 
2 2005-05-03  EA      2.0 
3 2005-05-04  EA      2.0 
4 2005-05-05  EA      2.0 
5 2005-05-06  EA      2.0 
6 2005-05-07  EA      2.0 
7 2005-05-08  EA      2.0 
8 2005-05-09  EA      2.0 
9 2005-05-10  EA      2.0 


In [18]:
# Check if 'Period' is in the columns; if not, the index should already be the 'Period'
if 'Period' in inflation_data_ea.columns:
    inflation_data_ea['Period'] = pd.to_datetime(inflation_data_ea['Period'])
    inflation_data_ea.set_index('Period', inplace=True)
elif not isinstance(inflation_data_ea.index, pd.DatetimeIndex):
    # If the index is not a datetime, convert it
    inflation_data_ea.index = pd.to_datetime(inflation_data_ea.index)

# Convert non-numeric values to NaN in the 'Inflation' column
inflation_data_ea['Inflation'] = pd.to_numeric(inflation_data_ea['Inflation'], errors='coerce')

# Create a daily date range that covers the entire period of your data
date_range = pd.date_range(start=inflation_data_ea.index.min(), end=inflation_data_ea.index.max(), freq='D')

# Create a new DataFrame for KNN imputation
inflation_data = pd.DataFrame(date_range, columns=['Period'])

# Merge with your existing data - this will introduce NaNs for days without data
inflation_data = inflation_data.merge(inflation_data_ea, how='left', left_on='Period', right_index=True)

# Convert 'Period' to a numerical value (e.g., days since start)
min_date = inflation_data['Period'].min()
inflation_data['Day_Ordinal'] = (inflation_data['Period'] - min_date).dt.days

# Prepare data for KNN imputation
knn_data = inflation_data[['Day_Ordinal', 'Inflation']]

# Initialize KNN Imputer - 5 neighbors / Week
knn_imputer = KNNImputer(n_neighbors=5)

# Apply KNN imputation
knn_data_imputed = knn_imputer.fit_transform(knn_data)

# Add imputed data back to DataFrame
inflation_data['Inflation'] = knn_data_imputed[:, 1]

# Fill 'geo' column with 'EA'
inflation_data['geo'] = 'EA'

# Convert 'Day_Ordinal' back to datetime
inflation_data['Period'] = inflation_data['Day_Ordinal'].apply(lambda x: min_date + timedelta(days=x))

# Drop the 'Day_Ordinal' column
inflation_data.drop(columns='Day_Ordinal', inplace=True)

# Display the first 10 rows of the daily data
print(inflation_data.head(10))

      Period geo  Inflation
0 2005-05-01  EA       2.00
1 2005-05-02  EA       2.18
2 2005-05-03  EA       2.18
3 2005-05-04  EA       2.18
4 2005-05-05  EA       2.18
5 2005-05-06  EA       2.18
6 2005-05-07  EA       2.18
7 2005-05-08  EA       2.18
8 2005-05-09  EA       2.18
9 2005-05-10  EA       2.18


# SparkSQL: Tables Connection

In [19]:
# Create a Spark session
spark = SparkSession.builder \
    .appName("Stock Market Trends Analysis") \
    .getOrCreate()

# Function to format date columns in Pandas DataFrame
def format_date_columns(df, date_column):
    df[date_column] = pd.to_datetime(df[date_column]).dt.date
    return df

# Format date columns
index_data = format_date_columns(index_data, 'date')
interest_data = format_date_columns(interest_data, 'Period')
bonds_data = format_date_columns(bonds_data, 'Period')
inflation_data = format_date_columns(inflation_data, 'Period')

# Convert the Pandas DataFrames to Spark DataFrames
index_df = spark.createDataFrame(index_data)
interest_df = spark.createDataFrame(interest_data)
bonds_df = spark.createDataFrame(bonds_data)
inflation_df = spark.createDataFrame(inflation_data)

# Trim and convert date columns to DateType
index_df = index_df.withColumn("date", to_date(trim(col("date")), "yyyy-MM-dd"))
interest_df = interest_df.withColumn("Period", to_date(trim(col("Period")), "yyyy-MM-dd"))
bonds_df = bonds_df.withColumn("Period", to_date(trim(col("Period")), "yyyy-MM-dd"))
inflation_df = inflation_df.withColumn("Period", to_date(trim(col("Period")), "yyyy-MM-dd"))

# Create Temporary Views
index_df.createOrReplaceTempView("indexView")
interest_df.createOrReplaceTempView("interestView")
bonds_df.createOrReplaceTempView("bondsView")
inflation_df.createOrReplaceTempView("inflationView")

# SQL Query for Joining
join_query = """
SELECT i.date, i.open, i.high, i.low, i.close, 
       int.Interest as InterestRate, 
       b.Rate as BondRate, b.maturity as BondMaturity, 
       inf.Inflation as InflationRate
FROM indexView i
LEFT JOIN interestView int ON i.date = int.Period
LEFT JOIN bondsView b ON i.date = b.Period
LEFT JOIN inflationView inf ON i.date = inf.Period
"""

# Execute the query
dataset = spark.sql(join_query)

# Show the result
dataset.show(10)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/13 11:42:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/01/13 11:42:49 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


+----------+------------+------------+------------+------------+------------+--------+------------+-------------+
|      date|        open|        high|         low|       close|InterestRate|BondRate|BondMaturity|InflationRate|
+----------+------------+------------+------------+------------+------------+--------+------------+-------------+
|2011-10-24|2346.8300781|     2372.25|2323.6899414|2369.0700684|     1.44908|  4.2126|          Y9|          2.9|
|2011-10-24|2346.8300781|     2372.25|2323.6899414|2369.0700684|     1.44908| 4.03609|          Y8|          2.9|
|2011-10-24|2346.8300781|     2372.25|2323.6899414|2369.0700684|     1.44908| 3.82462|          Y7|          2.9|
|2011-10-24|2346.8300781|     2372.25|2323.6899414|2369.0700684|     1.44908| 3.57604|          Y6|          2.9|
|2011-10-24|2346.8300781|     2372.25|2323.6899414|2369.0700684|     1.44908| 3.29719|          Y5|          2.9|
|2011-10-24|2346.8300781|     2372.25|2323.6899414|2369.0700684|     1.44908| 3.01348|  

In [20]:
# DataFrame: joined_df
numeric_column_types = ['int', 'double', 'float', 'long']

# Counting zeros and nulls for each column
for column in dataset.columns:
    # Check if the column is numeric
    if str(dataset.schema[column].dataType) in numeric_column_types:
        zero_count = dataset.filter(col(column) == 0).count()
        null_or_nan_count = joined_df.filter(col(column).isNull() | isnan(col(column))).count()
    else:
        zero_count = 'N/A' # Not applicable for non-numeric columns
        null_or_nan_count = dataset.filter(col(column).isNull()).count() # Check only for nulls in non-numeric columns

    print(f"Column '{column}': Zeros = {zero_count}, Nulls or NaNs = {null_or_nan_count}")

Column 'date': Zeros = N/A, Nulls or NaNs = 0
Column 'open': Zeros = N/A, Nulls or NaNs = 0
Column 'high': Zeros = N/A, Nulls or NaNs = 0
Column 'low': Zeros = N/A, Nulls or NaNs = 0


Column 'close': Zeros = N/A, Nulls or NaNs = 0


Column 'InterestRate': Zeros = N/A, Nulls or NaNs = 967
Column 'BondRate': Zeros = N/A, Nulls or NaNs = 17
Column 'BondMaturity': Zeros = N/A, Nulls or NaNs = 17
Column 'InflationRate': Zeros = N/A, Nulls or NaNs = 769


In [21]:
# Print the schema of the DataFrame
dataset.printSchema()

# Count the number of rows in the DataFrame
print("Number of rows:", dataset.count())

# Show statistical summaries of the DataFrame
dataset.describe().show()

root
 |-- date: date (nullable = true)
 |-- open: double (nullable = true)
 |-- high: double (nullable = true)
 |-- low: double (nullable = true)
 |-- close: double (nullable = true)
 |-- InterestRate: double (nullable = true)
 |-- BondRate: string (nullable = true)
 |-- BondMaturity: string (nullable = true)
 |-- InflationRate: double (nullable = true)



Number of rows: 42983


24/01/13 11:43:05 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+------------------+------------------+------------------+-----------------+------------------+------------------+------------+------------------+
|summary|              open|              high|               low|            close|      InterestRate|          BondRate|BondMaturity|     InflationRate|
+-------+------------------+------------------+------------------+-----------------+------------------+------------------+------------+------------------+
|  count|             42983|             42983|             42983|            42983|             42016|             42966|       42966|             42214|
|   mean|3318.0367247533245|3338.8286606793904|3295.4201961815234|3317.759078447996|1.0332473686214005| 1.745217309500524|        NULL|2.1377855687687473|
| stddev| 587.8283901504929| 585.8124922448433| 590.5245824997439|588.1531904750066| 1.655530792323179|1.5987436897765723|        NULL| 2.079979486913024|
|    min|      1812.7800293|           1823.25|      1765.4899902|    

In [22]:
# Count the total number of rows
total_rows = dataset.count()

# Count rows where specific columns are available but others are missing (Null or NaN)
# Adjust column names as per your DataFrame
missing_columns_count = dataset.filter(
    (col("InterestRate").isNull() | isnan(col("InterestRate"))) &
    (col("BondRate").isNull() | isnan(col("BondRate"))) &
    (col("InflationRate").isNull() | isnan(col("InflationRate"))) &
    col("open").isNotNull()
).count()

# Calculate the percentage of such instances
percentage_missing_columns = (missing_columns_count / total_rows) * 100

print(f"Total Rows: {total_rows}")
print(f"Rows with 'Open' but missing 'InterestRate', 'BondRate', and 'InflationRate': {missing_columns_count}")
print(f"Percentage of Rows with missing rates: {percentage_missing_columns:.2f}%")

# Filter the DataFrame to get rows with missing 'InterestRate', 'BondRate', and 'InflationRate' but available 'Open'
missing_values_table = dataset.filter(
    (col("InterestRate").isNull() | isnan(col("InterestRate"))) &
    (col("BondRate").isNull() | isnan(col("BondRate"))) &
    (col("InflationRate").isNull() | isnan(col("InflationRate"))) &
    col("open").isNotNull()
)

# Show the rows with missing values
missing_values_table.show()


Total Rows: 42983
Rows with 'Open' but missing 'InterestRate', 'BondRate', and 'InflationRate': 4
Percentage of Rows with missing rates: 0.01%


+----------+----------+----------+----------+----------+------------+--------+------------+-------------+
|      date|      open|      high|       low|     close|InterestRate|BondRate|BondMaturity|InflationRate|
+----------+----------+----------+----------+----------+------------+--------+------------+-------------+
|2024-01-11| 4481.3999|4510.00977|4441.45996|4442.27979|        NULL|    NULL|        NULL|         NULL|
|2023-12-29|4518.12012|4536.43994|4517.31006| 4521.6499|        NULL|    NULL|        NULL|         NULL|
|2023-12-22| 4520.6001| 4528.0498|4505.72998|4521.47021|        NULL|    NULL|        NULL|         NULL|
|2024-01-12|4453.24023|4492.77002|4453.24023|4480.02002|        NULL|    NULL|        NULL|         NULL|
+----------+----------+----------+----------+----------+------------+--------+------------+-------------+



In [23]:
# Count the total number of rows
total_rows = dataset.count()

# Count rows where any one of the specific columns is available but others are missing (Null or NaN)
# Adjust column names as per your DataFrame
missing_any_column_count = dataset.filter(
    col("open").isNotNull() &
    (
        col("InterestRate").isNull() | isnan(col("InterestRate")) |
        col("BondRate").isNull() | isnan(col("BondRate")) |
        col("InflationRate").isNull() | isnan(col("InflationRate"))
    )
).count()

# Calculate the percentage of such instances
percentage_missing_any_column = (missing_any_column_count / total_rows) * 100

print(f"Total Rows: {total_rows}")
print(f"Rows with 'Open' but missing any one of 'InterestRate', 'BondRate', or 'InflationRate': {missing_any_column_count}")
print(f"Percentage of Rows with missing any one rate: {percentage_missing_any_column:.2f}%")

# Filter the DataFrame to get rows with missing any one of 'InterestRate', 'BondRate', and 'InflationRate' but available 'Open'
missing_any_values_table = dataset.filter(
    col("open").isNotNull() &
    (
        col("InterestRate").isNull() | isnan(col("InterestRate")) |
        col("BondRate").isNull() | isnan(col("BondRate")) |
        col("InflationRate").isNull() | isnan(col("InflationRate"))
    )
)

# Show the rows with missing values
missing_any_values_table.show()


Total Rows: 42983
Rows with 'Open' but missing any one of 'InterestRate', 'BondRate', or 'InflationRate': 980
Percentage of Rows with missing any one rate: 2.28%
+----------+----------+----------+------------+------------+------------+--------+------------+-------------+
|      date|      open|      high|         low|       close|InterestRate|BondRate|BondMaturity|InflationRate|
+----------+----------+----------+------------+------------+------------+--------+------------+-------------+
|2022-06-22|    3494.0|    3494.0|3413.0400391|3464.6398926|     0.16346|    NULL|        NULL|         8.92|
|2024-01-11| 4481.3999|4510.00977|  4441.45996|  4442.27979|        NULL|    NULL|        NULL|         NULL|
|2024-01-02| 4528.4502| 4568.7998|  4484.81982|  4512.81006|        NULL| 2.74871|          Y9|         NULL|
|2024-01-02| 4528.4502| 4568.7998|  4484.81982|  4512.81006|        NULL| 2.67428|          Y8|         NULL|
|2024-01-02| 4528.4502| 4568.7998|  4484.81982|  4512.81006|        

## Window Approuch

In [35]:
from pyspark.sql import Window
from pyspark.sql.functions import last

# Define the window specification for forward fill with partitioning by 'date'
window_spec = Window.partitionBy("date").orderBy("date").rowsBetween(Window.unboundedPreceding, Window.currentRow)

# Apply forward fill for each column that needs imputation
dataset_ff = dataset.withColumn("InterestRate", last("InterestRate", ignorenulls=True).over(window_spec)) \
                    .withColumn("BondRate", last("BondRate", ignorenulls=True).over(window_spec)) \
                    .withColumn("InflationRate", last("InflationRate", ignorenulls=True).over(window_spec))

# Show the result
dataset_ff.show()

+----------+-----------+-----------+-----------+-----------+------------+--------+------------+-------------+
|      date|       open|       high|        low|      close|InterestRate|BondRate|BondMaturity|InflationRate|
+----------+-----------+-----------+-----------+-----------+------------+--------+------------+-------------+
|2005-02-01|3008.850098|3008.850098|3008.850098|3008.850098|        NULL| 3.48231|          Y9|         NULL|
|2005-02-01|3008.850098|3008.850098|3008.850098|3008.850098|        NULL| 3.37388|          Y8|         NULL|
|2005-02-01|3008.850098|3008.850098|3008.850098|3008.850098|        NULL| 3.25104|          Y7|         NULL|
|2005-02-01|3008.850098|3008.850098|3008.850098|3008.850098|        NULL| 3.11258|          Y6|         NULL|
|2005-02-01|3008.850098|3008.850098|3008.850098|3008.850098|        NULL| 2.95754|          Y5|         NULL|
|2005-02-01|3008.850098|3008.850098|3008.850098|3008.850098|        NULL| 2.78549|          Y4|         NULL|
|2005-02-0

In [36]:
from pyspark.sql.functions import col, isnan, count

# Count the total number of rows
total_rows = dataset_ff.count()

# Count rows where 'Open' is available but any one of 'InterestRate', 'BondRate', or 'InflationRate' is missing (Null or NaN)
missing_any_column_count = dataset_ff.filter(
    col("open").isNotNull() &
    (
        col("InterestRate").isNull() | isnan(col("InterestRate")) |
        col("BondRate").isNull() | isnan(col("BondRate")) |
        col("InflationRate").isNull() | isnan(col("InflationRate"))
    )
).count()

# Calculate the percentage of such instances
percentage_missing_any_column = (missing_any_column_count / total_rows) * 100

# Output the results
print(f"Total Rows: {total_rows}")
print(f"Rows with 'Open' but missing any one of 'InterestRate', 'BondRate', or 'InflationRate': {missing_any_column_count}")
print(f"Percentage of Rows with missing any one rate: {percentage_missing_any_column:.2f}%")

Total Rows: 42983
Rows with 'Open' but missing any one of 'InterestRate', 'BondRate', or 'InflationRate': 980
Percentage of Rows with missing any one rate: 2.28%


Here's why the Window function is used:

    Row-wise Calculations: PySpark's Window function allows you to specify a range of rows (a window) relative to the current row and perform calculations across these rows.

    Emulating ffill: By ordering the rows in the window based on the 'date' column and then using the last function (which returns the last non-null value in the window), we effectively emulate the forward fill behavior. It picks the last non-null value encountered in the ordered window and applies it to the current row if the current value is null.

    Scalability and Performance: PySpark is designed for big data processing. While Pandas' ffill is straightforward and efficient for in-memory data on a single machine, PySpark's window functions are designed to work efficiently on distributed datasets across multiple nodes in a cluster.

    Lack of Direct ffill Equivalent: PySpark DataFrame API does not have a direct method like ffill in Pandas. Therefore, the combination of Window and last functions is a common approach to achieve similar functionality.

### Because of the Nature of the Fields (Interest Rate and Inflation Rate), is possible to use the MEAN to fill the Missing Day Value

In [37]:
from pyspark.sql.functions import mean

# Calculate the mean of non-null values for each column
mean_values = dataset.select([mean(c).alias(c) for c in numeric_cols]).collect()[0].asDict()

# Fill null values with the mean
for col in numeric_cols:
    dataset_ff = dataset_ff.na.fill({col: mean_values[col]})

# Show the result
dataset_ff.show()


+----------+-----------+-----------+-----------+-----------+------------------+--------+------------+------------------+
|      date|       open|       high|        low|      close|      InterestRate|BondRate|BondMaturity|     InflationRate|
+----------+-----------+-----------+-----------+-----------+------------------+--------+------------+------------------+
|2005-02-01|3008.850098|3008.850098|3008.850098|3008.850098|1.0332473686214005| 3.48231|          Y9|2.1377855687687473|
|2005-02-01|3008.850098|3008.850098|3008.850098|3008.850098|1.0332473686214005| 3.37388|          Y8|2.1377855687687473|
|2005-02-01|3008.850098|3008.850098|3008.850098|3008.850098|1.0332473686214005| 3.25104|          Y7|2.1377855687687473|
|2005-02-01|3008.850098|3008.850098|3008.850098|3008.850098|1.0332473686214005| 3.11258|          Y6|2.1377855687687473|
|2005-02-01|3008.850098|3008.850098|3008.850098|3008.850098|1.0332473686214005| 2.95754|          Y5|2.1377855687687473|
|2005-02-01|3008.850098|3008.850

In [38]:
from pyspark.sql.functions import col, isnan, count

# Count the total number of rows
total_rows = dataset_ff.count()

# Count rows where 'Open' is available but any one of 'InterestRate', 'BondRate', or 'InflationRate' is missing (Null or NaN)
missing_any_column_count = dataset_ff.filter(
    col("open").isNotNull() &
    (
        col("InterestRate").isNull() | isnan(col("InterestRate")) |
        col("BondRate").isNull() | isnan(col("BondRate")) |
        col("InflationRate").isNull() | isnan(col("InflationRate"))
    )
).count()

# Calculate the percentage of such instances
percentage_missing_any_column = (missing_any_column_count / total_rows) * 100

# Output the results
print(f"Total Rows: {total_rows}")
print(f"Rows with 'Open' but missing any one of 'InterestRate', 'BondRate', or 'InflationRate': {missing_any_column_count}")
print(f"Percentage of Rows with missing any one rate: {percentage_missing_any_column:.2f}%")

Total Rows: 42983
Rows with 'Open' but missing any one of 'InterestRate', 'BondRate', or 'InflationRate': 0
Percentage of Rows with missing any one rate: 0.00%


## KNN Approuch

In [25]:
# Convert Spark DataFrame to Pandas DataFrame
dataset_pd = dataset.toPandas()

# Replace non-numeric values to NaN
numeric_cols = ["InterestRate", "BondRate", "InflationRate"]
dataset_pd[numeric_cols] = dataset_pd[numeric_cols].apply(pd.to_numeric, errors='coerce')

# Apply KNN imputation
knn_imputer = KNNImputer(n_neighbors=5)
dataset_pd[numeric_cols] = knn_imputer.fit_transform(dataset_pd[numeric_cols])

# Convert back to Spark DataFrame
dataset_knn = spark.createDataFrame(dataset_pd)


In [29]:
# Sort the DataFrame in ascending order by the 'date' column
dataset_knn = dataset_knn.orderBy("date")

# Show the result
dataset_knn.show()


+----------+-----------+-----------+-----------+-----------+------------------+--------+------------+------------------+
|      date|       open|       high|        low|      close|      InterestRate|BondRate|BondMaturity|     InflationRate|
+----------+-----------+-----------+-----------+-----------+------------------+--------+------------+------------------+
|2005-02-01|3008.850098|3008.850098|3008.850098|3008.850098|1.9096960000000003| 3.11258|          Y6|2.8160000000000003|
|2005-02-01|3008.850098|3008.850098|3008.850098|3008.850098|0.7373880000000002| 2.78549|          Y4|1.6800000000000002|
|2005-02-01|3008.850098|3008.850098|3008.850098|3008.850098|             1.232| 3.25104|          Y7|             1.488|
|2005-02-01|3008.850098|3008.850098|3008.850098|3008.850098|          2.504148| 3.48231|          Y9|1.7560000000000002|
|2005-02-01|3008.850098|3008.850098|3008.850098|3008.850098|          2.760096| 3.37388|          Y8|3.8560000000000003|
|2005-02-01|3008.850098|3008.850

In [39]:
from pyspark.sql.functions import col, isnan, count

# Count the total number of rows
total_rows = dataset_knn.count()

# Count rows where 'Open' is available but any one of 'InterestRate', 'BondRate', or 'InflationRate' is missing (Null or NaN)
missing_any_column_count = dataset_knn.filter(
    col("open").isNotNull() &
    (
        col("InterestRate").isNull() | isnan(col("InterestRate")) |
        col("BondRate").isNull() | isnan(col("BondRate")) |
        col("InflationRate").isNull() | isnan(col("InflationRate"))
    )
).count()

# Calculate the percentage of such instances
percentage_missing_any_column = (missing_any_column_count / total_rows) * 100

# Output the results
print(f"Total Rows: {total_rows}")
print(f"Rows with 'Open' but missing any one of 'InterestRate', 'BondRate', or 'InflationRate': {missing_any_column_count}")
print(f"Percentage of Rows with missing any one rate: {percentage_missing_any_column:.2f}%")

Total Rows: 42983
Rows with 'Open' but missing any one of 'InterestRate', 'BondRate', or 'InflationRate': 0
Percentage of Rows with missing any one rate: 0.00%
